In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys;src_dir = os.path.join(os.getcwd(), os.pardir, 'src');sys.path.append(src_dir)

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import features.smooth as sm
import features.angles as ang
import sklearn.preprocessing
import tqdm
import tsfresh
#import tsfresh
from data.dal import DAL
features = DAL("features.hdf")
split = DAL("split.hdf")
features.overview()

/export/home1/NoCsBack/dtai/tomd/miniconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,df_key,description
0,rawconcat,Raw coordinates (all split files concatenated)
1,angles_raw,angles computed of rawconcat


## Smoothening with Fourier transform and flattening exercises into 1 dataframe

In [40]:
split = DAL("split.hdf")
ov = split.overview()

def fouriertransform(overview):
    frames = []
    for row in tqdm.tqdm(list(overview.itertuples())):
        df = split.get(row.df_key)
        df = sm.fourier(df,5)
        df['id'] = row.Index
        frames.append(df)
    return pd.concat(frames)
fourier5 = fouriertransform(ov)
features.add(fourier5,"fourier5","Raw coordinates smoothed to their 5 highest fourier freqencies")

100%|██████████| 1790/1790 [00:30<00:00, 58.21it/s]


In [3]:
#no fourier
split = DAL("split.hdf")
ov = split.overview()

def concat(overview):
    frames = []
    for row in tqdm.tqdm(list(overview.itertuples())):
        df = split.get(row.df_key)
        #df = sm.fourier(df,5)
        df['id'] = row.Index
        frames.append(df)
    return pd.concat(frames)
c = concat(ov)
features.add(c,"rawconcat","Raw coordinates (all split files concatenated)")

100%|██████████| 1790/1790 [00:06<00:00, 265.03it/s]


## Computing angles

In [4]:
df = features.get("rawconcat")
angdf = ang.kinect.get_angles(df)
angdf['id'] = df['id']
features.add(angdf,"angles_raw","angles computed of rawconcat")
features.overview()

,df_key,description
0,rawconcat,Raw coordinates (all split files concatenated)
1,angles_raw,angles computed of rawconcat


## computing tsfresh features

In [7]:
df = features.get("angles_raw")
settings = tsfresh.feature_extraction.settings.MinimalFCParameters()
tsfreshfeatures = tsfresh.extract_features(df,column_id="id",default_fc_parameters=settings)
features.add(tsfreshfeatures,"tsfresh_raw_min", "Minimal tsfresh features on angles_raw")

Feature Extraction: 100%|██████████| 20/20 [00:04<00:00,  4.97it/s]
